# OpenClassrooms - Ingenieur IA
# Projet 8 - Future Vision Transport 
# Participez à la conception d'une voiture autonome

## Objectif du projet : 
- **Concevoir un premier modèle de segmentation d’images**

## Plan :
- **Mise en place de l'environnement de travail dans Azure**
    - Récupération du Workspace Azure
    - Récupération des données préparées du Datastore Azure
    - Création d'un Dataset Azure
    - Enregistrement du Dataset dans Azure
    - Création de l'environnement Python dans Azure
    - Enregistrement de l'environnement Python dans Azure

In [2]:
import os

from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

from azureml.core.datastore import Datastore
from azureml.core.dataset import Dataset
from azureml.core.authentication import ServicePrincipalAuthentication

from azureml.data.dataset_type_definitions import FileType

import warnings
warnings.filterwarnings('ignore')

# Mise en place de l'environnement de travail dans Azure

### Récupération du Workspace Azure : authentification utilisant le service recommandé 'Service Principal Authentication'

In [3]:
svc_pr_password = os.environ['CITYSCAPE_SPA_PASSWORD']
tenant_id = '894ad120-c276-4dfa-b218-d82b3fece6a7'
application_id = '21cf902f-1dc0-459d-b352-b7490946f6c6'

In [4]:
svc_pr = ServicePrincipalAuthentication(
        tenant_id=tenant_id,
        service_principal_id=application_id,
        service_principal_password=svc_pr_password)

In [5]:
subscription_id = subscription_id = os.environ['AZURE_SUBSCRIPTION_ID']
resource_group = 'OC-IA-P8-GPU'
workspace_name = 'WS-IA-P8-GPU'
ws = Workspace(
        subscription_id=subscription_id,
        resource_group=resource_group,
        workspace_name=workspace_name,
        auth=svc_pr)

In [6]:
print(ws.name, ws.location, ws.resource_group, sep='\t')

WS-IA-P8-GPU	eastus	OC-IA-P8-GPU


## Récupération des données préparées du Datastore Azure

In [7]:
datastore_prepared_name = "datastore_cityscape_prepared"

In [8]:
datastore_prepared = Datastore.get(ws, datastore_prepared_name)

In [9]:
datastore_prepared

{
  "name": "datastore_cityscape_prepared",
  "container_name": "cityscape-prepared",
  "account_name": "wsiap8gpu5520371261",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

## Création d'un Dataset Azure

In [10]:
dataset_prepared = Dataset.File.from_files(path=[(datastore_prepared, ('/prepared_data/'))])

In [11]:
dataset_prepared

{
  "source": [
    "('datastore_cityscape_prepared', 'prepared_data/')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ]
}

### Sanity check Dataset

In [12]:
dataset_prepared.to_path()[0]

'/images/test/0.png'

In [13]:
len(dataset_prepared.to_path())

7070

## Enregistrement du Dataset dans Azure (s'il n'est pas déjà enregistré)

In [14]:
dataset_prepared_name = "dataset_cityscape_prepared"

In [15]:
dataset_registered = False

try:
    temp = Dataset.get_by_name(workspace = ws, name = dataset_prepared_name)
    dataset_registered = True
    print("Le dataset pour le projet Cityscape est déjà enregistré.")
except:
    print("Le dataset pour le projet Cityscape n'est pas encore enregistré.")

if not dataset_registered:
    dataset_prepared.register(workspace = ws,
                    name = dataset_prepared_name,
                    description='Dataset pour le projet Cityscape',
                    create_new_version=False)
    print("Le dataset pour le projet Cityscape a bien été enregistré.")

Le dataset pour le projet Cityscape est déjà enregistré.


## Création de l'environnement Python dans Azure
- On récupère un environement Python dans Azure existant incorporant TensorFlow et la gestion des GPU
- On ajoute à cet environnement les modules dont nous avons besoin
- On supprime de cet environnement les modules dont nous n'avons pas besoin

In [16]:
environment_name="cityscape-environment"

In [17]:
environment_list = Environment.list(workspace=ws)
environment = environment_list['AzureML-TensorFlow-2.3-GPU'].clone(environment_name)
environment.docker.enabled = True
conda_dep = environment.python.conda_dependencies
conda_dep.add_conda_package("pip")
conda_dep.add_conda_package("pillow")
conda_dep.add_pip_package("opencv-python")
conda_dep.add_pip_package("albumentations")
conda_dep.add_pip_package("segmentation-models")
conda_dep.remove_pip_package('horovod')
conda_dep.remove_pip_package('cmake')
environment.python.conda_dependencies=conda_dep
environment

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04:20210922.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "cityscape-environment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
     

## Enregistrement de l'environnement Python dans Azure (s'il n'est pas déjà enregistré)

In [18]:
environment_registered = False

try:
    temp = Environment.get(workspace = ws, name = environment_name)
    environment_registered = True
    print("L'environment Python pour le projet Cityscape est déjà enregistré.")
except:
    print("L'environment Python pour le projet Cityscape n'est pas encore enregistré.")

if not environment_registered:
    environment.register(workspace = ws)
    print("L'environment Python pour le projet Cityscape a bien été enregistré.")

L'environment Python pour le projet Cityscape est déjà enregistré.
